<a href="https://colab.research.google.com/github/BuczynskiRafal/BuczynskiRafal/blob/main/ANN/05_classification_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### <a name='a0'></a> 1. Import bibliotek

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### <a name='a1'></a> 2. Metryki - Klasyfikacja binarna
#### <a name='a2'></a> 2.1 Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [2]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

In [3]:
def accuracy(y_true, y_pred):
    correct = 0
    for idx, _ in enumerate(y_true):
        if y_true[idx] == y_pred[idx]:
            correct += 1
    return correct / len(y_true) * 100

In [5]:
accuracy(y_true, y_pred)

71.42857142857143

In [6]:
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results = results.sort_values(by='y_true')
results['sample'] = range(1, len(y_true) + 1)
results

,y_true,y_pred,sample
1,0,0,1
4,0,0,2
7,0,0,3
9,0,1,4
12,0,0,5
0,1,0,6
2,1,1,7
3,1,1,8
5,1,1,9
6,1,0,10


In [7]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1, col=1)
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
fig.update_layout(width=1000, height=800, title='Binary classifier predictions')
fig.show()

#### <a name='a3'></a> 2.2 Macierz konfuzji/pomyłek

In [8]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
cm

array([[4, 1],
       [3, 6]])

In [9]:
import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

In [10]:
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
cm_df

,pred_0,pred_1
true_0,4,1
true_1,3,6


In [11]:
tn, fp, fn, tp = cm.ravel()
print(f'TN - True Negative: {tn}')
print(f'FP - False Positive: {fp}')
print(f'FN - False Negative: {fn}')
print(f'TP - True Positive: {tp}')

TN - True Negative: 4
FP - False Positive: 1
FN - False Negative: 3
TP - True Positive: 6


In [12]:
# False Positive Rate - Type I error
fpr = fp / (fp + tn)
print(fpr)

0.2


In [13]:
# False Negative Rate - Type II error
fnr = fn / (fn + tp)
print(fnr)

0.3333333333333333


In [14]:
# Precision - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne
precision = tp / (tp + fp)
print(precision)

0.8571428571428571


In [15]:
# Recall - jak wiele obserwacji z wzystkich poytywnych sklasyfikowaliśmy jako pozytywne
recall = tp / (tp + fn)
print(recall)

0.6666666666666666


#### <a name='a4'></a> 2.3 Krzywa ROC

In [16]:
# Krzywa ROC
from sklearn.metrics import roc_curve
fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

In [17]:
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc

,fpr,tpr
0,0.0,0.000000
1,0.2,0.666667
2,1.0,1.000000


In [18]:
def plot_roc_curve(y_true, y_pred):
    # Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

    fig = go.Figure(data=[go.Scatter(x=roc['fpr'], y=roc['tpr'], line_color='red', name='ROC Curve'),
                        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line_dash='dash', line_color='navy')],
                    layout=go.Layout(xaxis_title='False Positive Rate',
                                    yaxis_title='True Positive Rate',
                                    title='ROC Curve',
                                    showlegend=False,
                                    width=800))
    fig.show()
plot_roc_curve(y_true, y_pred)

In [19]:
# AUC Score
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_true, y_pred)
roc_auc

0.7333333333333333

In [20]:
# Gini coefficient
gini = roc_auc * 2 - 1
print(gini)

0.46666666666666656


#### <a name='a5'></a> 3. Metryki - Klasyfikacja wieloklasowa

#### <a name='a6'></a> 3.1 Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [21]:
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2])

from sklearn.metrics import accuracy_score

accuracy_score(y_true, y_pred)

0.6842105263157895

#### <a name='a7'></a> 3.2 Macierz konfuzji/pomyłek

In [22]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[4, 1, 0],
       [3, 6, 1],
       [0, 1, 3]])

In [23]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1', 'pred_2'], index=['true_2','true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

#### <a name='a8'></a> 3.3 Raport klasyfikacji

In [25]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.80      0.67         5
           1       0.75      0.60      0.67        10
           2       0.75      0.75      0.75         4

    accuracy                           0.68        19
   macro avg       0.69      0.72      0.69        19
weighted avg       0.70      0.68      0.68        19

